# COMP5318 Assignment 1: Rice Classification

##### Group number: Set 1, Group 153
##### Student 1 SID: 550403876
##### Student 2 SID: ...  

In [78]:
# Import all libraries
from sklearn.model_selection import StratifiedKFold

In [79]:
# Ignore future warnings
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

In [80]:
# Load the rice dataset: rice-final2.csv
import pandas as pd
import numpy as np

data = pd.read_csv('rice-final2.csv')
# data = pd.read_csv('test/test-before.csv')

In [81]:
# Pre-process dataset
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

#Map to numeric values in target class
y = data['class'].map({'class1': 0, 'class2': 1})  

# Convert feature columns to numeric, swapping errors with NaN
X = data.drop(columns='class')
for col in X.columns:
    X[col] = pd.to_numeric(data[col], errors='coerce')

# Impute missing values with mean
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
imp_mean.fit(X)
X = imp_mean.transform(X)

# Normalize
scaler = MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X)


In [82]:
# Print first ten rows of pre-processed dataset to 4 decimal places as per assignment spec
# A function is provided to assist

def print_data(X, y, n_rows=10):
    """Takes a numpy data array and target and prints the first ten rows.
    
    Arguments:
        X: numpy array of shape (n_examples, n_features)
        y: numpy array of shape (n_examples)
        n_rows: numpy of rows to print
    """
    for example_num in range(n_rows):
        for feature in X[example_num]:
            print("{:.4f}".format(feature), end=",")

        if example_num == len(X)-1:
            print(y[example_num],end="")
        else:
            print(y[example_num])

print_data(X, y)

0.4628,0.5406,0.5113,0.4803,0.7380,0.4699,0.1196,1
0.4900,0.5547,0.5266,0.5018,0.7319,0.4926,0.8030,1
0.6109,0.6847,0.6707,0.5409,0.8032,0.6253,0.1185,0
0.6466,0.6930,0.6677,0.5961,0.7601,0.6467,0.2669,0
0.6712,0.6233,0.4755,0.8293,0.3721,0.6803,0.4211,1
0.2634,0.2932,0.2414,0.4127,0.5521,0.2752,0.2825,1
0.8175,0.9501,0.9515,0.5925,0.9245,0.8162,0.0000,0
0.3174,0.3588,0.3601,0.3908,0.6921,0.3261,0.8510,1
0.3130,0.3050,0.2150,0.5189,0.3974,0.3159,0.4570,1
0.5120,0.5237,0.4409,0.6235,0.5460,0.5111,0.3155,1


### Part 1: Cross-validation without parameter tuning

In [83]:
## Setting the 10 fold stratified cross-validation
cvKFold=StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

# The stratified folds from cvKFold should be provided to the classifiers

In [84]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# Logistic Regression
def logregClassifier(X, y):
    model = LogisticRegression(random_state=0)
    scores = cross_val_score(model, X, y, cv=cvKFold, scoring='accuracy')
    return scores.mean()

In [85]:
from sklearn.naive_bayes import GaussianNB

#Naïve Bayes
def nbClassifier(X, y):
    model = GaussianNB()
    scores = cross_val_score(model, X, y, cv=cvKFold, scoring='accuracy')
    return scores.mean()


In [86]:
from sklearn.tree import DecisionTreeClassifier

# Decision Tree
def dtClassifier(X, y):
    model = DecisionTreeClassifier(criterion='entropy',random_state=0)
    scores = cross_val_score(model, X, y, cv=cvKFold, scoring='accuracy')
    return scores.mean()

In [87]:
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier

# Ensembles: Bagging, Ada Boost and Gradient Boosting
def bagDTClassifier(X, y, n_estimators, max_samples, max_depth):
    model = BaggingClassifier(DecisionTreeClassifier(max_depth=max_depth, criterion='entropy', random_state=0),
                      n_estimators=n_estimators, max_samples=max_samples, random_state=0)
    scores = cross_val_score(model, X, y, cv=cvKFold, scoring='accuracy')
    return scores.mean()

def adaDTClassifier(X, y, n_estimators, learning_rate, max_depth):
    model = AdaBoostClassifier(DecisionTreeClassifier(max_depth=max_depth, criterion='entropy', random_state=0),
                      n_estimators=n_estimators, learning_rate=learning_rate, random_state=0)
    scores = cross_val_score(model, X, y, cv=cvKFold, scoring='accuracy')
    return scores.mean()

def gbClassifier(X, y, n_estimators, learning_rate):
    model = GradientBoostingClassifier(n_estimators=n_estimators, learning_rate=learning_rate, random_state=0)
    scores = cross_val_score(model, X, y, cv=cvKFold, scoring='accuracy')
    return scores.mean()

### Part 1 Results

In [88]:
# Parameters for Part 1:

#Bagging
bag_n_estimators = 50
bag_max_samples = 100
bag_max_depth = 5

#AdaBoost
ada_n_estimators = 50
ada_learning_rate = 0.5
ada_bag_max_depth = 5

#GB
gb_n_estimators = 50
gb_learning_rate = 0.5

# Print results for each classifier in part 1 to 4 decimal places here:
print("LogR average cross-validation accuracy: {:.4f}".format(logregClassifier(X,y)))
print("NB average cross-validation accuracy: {:.4f}".format(nbClassifier(X,y)))
print("DT average cross-validation accuracy: {:.4f}".format(dtClassifier(X,y)))
print("Bagging average cross-validation accuracy: {:.4f}".format(bagDTClassifier(X,y,bag_n_estimators,bag_max_samples,bag_max_depth)))
print("AdaBoost average cross-validation accuracy: {:.4f}".format(adaDTClassifier(X,y,ada_n_estimators,ada_learning_rate,ada_bag_max_depth)))
print("GB average cross-validation accuracy: {:.4f}".format(gbClassifier(X,y,gb_n_estimators,gb_learning_rate)))

LogR average cross-validation accuracy: 0.9386
NB average cross-validation accuracy: 0.9264
DT average cross-validation accuracy: 0.9179
Bagging average cross-validation accuracy: 0.9414
AdaBoost average cross-validation accuracy: 0.9407
GB average cross-validation accuracy: 0.9321


### Part 2: Cross-validation with parameter tuning

In [89]:
# KNN
k = [1, 3, 5, 7]
p = [1, 2]


def bestKNNClassifier(X, y):
    
    return #(appropriate values so that the required printing can be done)

In [90]:
# SVM
# You should use SVC from sklearn.svm with kernel set to 'rbf'
C = [0.01, 0.1, 1, 5] 
gamma = [0.01, 0.1, 1, 10]

def bestSVMClassifier(X, y):

    return  #(appropriate values so that the required printing can be done)

In [91]:
# Random Forest
# You should use RandomForestClassifier from sklearn.ensemble with information gain and max_features set to ‘sqrt’.
n_estimators = [10, 30, 60, 100]
max_leaf_nodes = [6, 12]

def bestRFClassifier(X, y):

    return #(appropriate values so that the required printing can be done)

### Part 2: Results

In [92]:
# Perform Grid Search with 10-fold stratified cross-validation (GridSearchCV in sklearn). 
# The stratified folds from cvKFold should be provided to GridSearchV

# This should include using train_test_split from sklearn.model_selection with stratification and random_state=0
# Print results for each classifier here. All results should be printed to 4 decimal places except for
# "k", "p", n_estimators" and "max_leaf_nodes" which should be printed as integers.
print("KNN best k: ")
print("KNN best p: ")
print("KNN cross-validation accuracy: ")
print("KNN test set accuracy: ")

print()

print("SVM best C: ")
print("SVM best gamma: ")
print("SVM cross-validation accuracy: ")
print("SVM test set accuracy: ")

print()

print("RF best n_estimators: ")
print("RF best max_leaf_nodes: ")
print("RF cross-validation accuracy: ")
print("RF test set accuracy: ")
print("RF test set macro average F1: ")
print("RF test set weighted average F1: ")

KNN best k: 
KNN best p: 
KNN cross-validation accuracy: 
KNN test set accuracy: 

SVM best C: 
SVM best gamma: 
SVM cross-validation accuracy: 
SVM test set accuracy: 

RF best n_estimators: 
RF best max_leaf_nodes: 
RF cross-validation accuracy: 
RF test set accuracy: 
RF test set macro average F1: 
RF test set weighted average F1: 


### Part 3: Reflection

##### Write one paragraph describing the most important thing that you have learned throughout this assignment.
##### Student 1: ...
##### Student 2: ...